## MVO Weights
4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,
2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,
7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245


In [1]:
import pandas as pd
import numpy as np
import random
import math
from datetime import date,timedelta

In [2]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights

In [3]:
def ratio(a,b,c):                       
  #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  #calculating sharpe ratio
  return (a-c)/b

In [4]:
# def number_of_years(y):#calculates the number of years of the dataset
#   p=y.index[0]         #date of first row in the dataset (datetime format)
#   q=y.index[len(y)-1]  #date of last row in the dataset  (datetime format)
#   return ((q-p).days+1)/365

In [5]:
df=pd.read_csv("2021-01-05.csv",parse_dates=['Date'],index_col='Date')  #Importing Dataset
#df = df.loc["2016-01-01" :"2020-12-31" ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)
col=list(df.columns)
tdf.shape

(3, 24)

In [6]:
end=tdf.index[2].date()

In [7]:
trading_days=250#len(df)/number_of_years(tdf) 

In [8]:
returnsh=df.pct_change()                  
#Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)

In [9]:
returnsh=returnsh.iloc[1:,:]
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,-0.007872,-0.012111,0.003190,-0.004176,0.003429,0.033265,0.031457,-0.006351,0.066918,0.016314,...,-0.003943,-0.005541,0.001686,0.037975,0.004156,0.077593,0.024541,-0.004940,0.006880,0.021386
2,0.014580,-0.018634,-0.008754,-0.000507,-0.005217,-0.003266,0.011169,0.007556,-0.015729,0.009912,...,0.009811,-0.002388,-0.012432,0.017618,0.009189,-0.017965,0.001896,0.012928,0.002628,0.024975


In [10]:
returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
  for i in range(1, len(df)):
    if returnso[cols][i] > 0:
      returnso[cols][i] = 0


In [11]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,-0.007872,-0.012111,0.000000,-0.004176,0.000000,0.000000,0.0,-0.006351,0.000000,0.0,...,-0.003943,-0.005541,0.000000,0.0,0.0,0.000000,0.0,-0.00494,0.0,0.0
2,0.000000,-0.018634,-0.008754,-0.000507,-0.005217,-0.003266,0.0,0.000000,-0.015729,0.0,...,0.000000,-0.002388,-0.012432,0.0,0.0,-0.017965,0.0,0.00000,0.0,0.0


In [12]:
covmatsh=returnsh.cov()*trading_days     
#Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso = returnso.cov() * trading_days

In [13]:
covmatsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
ASIANPAINT,0.063015,-0.018307,-0.033523,0.010299,-0.024267,-0.102529,-0.056941,0.039032,-0.231958,-0.017968,...,0.038601,0.008849,-0.039622,-0.057133,0.014124,-0.268193,-0.063555,0.050149,-0.011934,0.010072
BAJFINANCE,-0.018307,0.005319,0.009739,-0.002992,0.007050,0.029787,0.016542,-0.011339,0.067388,0.005220,...,-0.011214,-0.002571,0.011511,0.016598,-0.004103,0.077915,0.018464,-0.014569,0.003467,-0.002926
BAJAJFINSV,-0.033523,0.009739,0.017834,-0.005479,0.012910,0.054544,0.030292,-0.020764,0.123398,0.009559,...,-0.020535,-0.004708,0.021078,0.030394,-0.007514,0.142674,0.033810,-0.026678,0.006348,-0.005358
BRITANNIA,0.010299,-0.002992,-0.005479,0.001683,-0.003966,-0.016758,-0.009307,0.006379,-0.037912,-0.002937,...,0.006309,0.001446,-0.006476,-0.009338,0.002308,-0.043834,-0.010388,0.008196,-0.001950,0.001646
DIVISLAB,-0.024267,0.007050,0.012910,-0.003966,0.009345,0.039484,0.021928,-0.015031,0.089328,0.006920,...,-0.014865,-0.003408,0.015258,0.022002,-0.005439,0.103282,0.024475,-0.019312,0.004596,-0.003879
GRASIM,-0.102529,0.029787,0.054544,-0.016758,0.039484,0.166820,0.092646,-0.063506,0.377407,0.029235,...,-0.062805,-0.014398,0.064467,0.092958,-0.022980,0.436363,0.103407,-0.081595,0.019417,-0.016388
HCLTECH,-0.056941,0.016542,0.030292,-0.009307,0.021928,0.092646,0.051452,-0.035269,0.209599,0.016236,...,-0.034880,-0.007996,0.035802,0.051625,-0.012762,0.242340,0.057429,-0.045315,0.010783,-0.009101
HDFCBANK,0.039032,-0.011339,-0.020764,0.006379,-0.015031,-0.063506,-0.035269,0.024176,-0.143675,-0.011129,...,0.023909,0.005481,-0.024542,-0.035388,0.008748,-0.166118,-0.039366,0.031062,-0.007392,0.006239
HINDALCO,-0.231958,0.067388,0.123398,-0.037912,0.089328,0.377407,0.209599,-0.143675,0.853832,0.066140,...,-0.142088,-0.032574,0.145847,0.210304,-0.051990,0.987210,0.233944,-0.184596,0.043927,-0.037075
HINDUNILVR,-0.017968,0.005220,0.009559,-0.002937,0.006920,0.029235,0.016236,-0.011129,0.066140,0.005123,...,-0.011006,-0.002523,0.011298,0.016291,-0.004027,0.076472,0.018122,-0.014299,0.003403,-0.002872


In [14]:
risk_free_rate = 0.0358 #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [15]:
df
stocks=df.shape[1]
stocks

24

# Sharpe


In [16]:
global_warr_sortino=[]
global_war_sharpe=[]
sharpe_portfolio_return=[]
sharpe_portfolio_risk=[]
sharpe_portfolio_shratio=[]
sharpe_portfolio_stockWeights=[]

In [17]:

fitness=0
#for each iteration

weights=np.array(BALANCE(np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varsh=np.dot(weights.T,np.dot(covmatsh,weights))   

#portfolio risk
volatility_temp = np.sqrt(varsh)      

#Calculating fitness value(ie sharpe ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sharpe_pbest)


sharpe_portfolio_return.append(returns_temp)
sharpe_portfolio_risk.append(volatility_temp)
sharpe_portfolio_shratio.append(fitness)
sharpe_portfolio_stockWeights.append(weights)



In [18]:
sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  

for counter,symbol in enumerate(df.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)
sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
sharpe_optimal=pd.DataFrame(sharpe_optimal)
sharpe_optimal.to_csv("sharpe_optimal.csv")

sharpe_pc

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,1.404203,0.115201,11.878402,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [19]:
sharpe_pc.to_csv(str(end)+'MVOsharpe_A.csv')

# Sortino

In [20]:
global_warr_sortino=[]
sortino_portfolio_return=[]
sortino_portfolio_risk=[]
sortino_portfolio_soratio=[]
sortino_portfolio_stockWeights=[]

In [21]:

fitness=0
#for each iteration


#Making sure that the total amount of pheromone equals 1 
weights=np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varso=np.dot(weights.T,np.dot(covmatso,weights))   

#portfolio risk
volatility_temp = np.sqrt(varso)      

#Calculating fitness value(ie sortino ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sortino_pbest)

sortino_portfolio_return.append(returns_temp)
sortino_portfolio_risk.append(volatility_temp)
sortino_portfolio_soratio.append(fitness)
sortino_portfolio_stockWeights.append(weights)


In [22]:
sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  

for counter,symbol in enumerate(df.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
sortino_pc = pd.DataFrame(sortino_portfolio)
sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
sortino_optimal=pd.DataFrame(sortino_optimal)
sortino_optimal.to_csv("sortino_optimal.csv")
sortino_pc


,Returns,Semi Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,140.420331,3.315768,42.338468,4.576064,9.676986,2.881647,3.762807,6.100055,1.147082,1.639964,...,9.762288,2.014192,7.839004,7.728703,2.277867,3.332264,1.031637,7.217097,0.606272,1.736266


In [23]:

sortino_pc.to_csv(str(end)+'MVOsortino_A.csv')